# Slice Builder: GraphML Parsing

This works in tandem with the Graph Analyzer code previously written, if desired (not required). That code can be found here: https://github.com/pjw7904/Graph-Analyzer/tree/develop

Any graphml file comprised of basic node and edge tags will be able to work with this parser. Each node will be set up as a FABRIC node and any edge will be set up as a l2network between the specified nodes. Currently, this code does not consider extended LANs with more than two nodes.

This was written to take advantage of existing graphml files, as opposed to the FABRIC-enhanced graphml RSPEC files that contain hardware properties that go beyond the basic topological information.

## Input Required Information

| Variable | Use |
| --- | --- |
| SLICE_NAME    | Name of slice you want to create. Please make sure a slice with that name does not already exist. |
| SITE_NAME     | Name of the FABRIC site you want the nodes to be reserved at. This code does not consider inter-site situations, the entire topology is reserved on a single slice. |
| GRAPH_PATH    | Path to the graphml file you want to use to create a topology. |
| HAS_CLIENTS   | Enter True if clients are present in topology, if not, False. These nodes and the networks connecting them utilizes alternative naming and addressing structures. |
| CLIENT_PREFIX | The naming prefix given to each node (currently, this is required if the topology does have clients) |
| MEAS_ADD      | Enter True if measurements are to be taken on the slice. This requires the inclusion of a separate measurement node |

In [4]:
SLICE_NAME = "test2"
SITE_NAME = "AMST"
GRAPH_PATH = "/home/fabric/work/FABRIC-EIBP/graphs/eibp_small.graphml"
HAS_CLIENTS = True
CLIENT_PREFIX = "ipnode"
MEAS_ADD = True

## Import the FABlib Library and Confirm the Configuration is Correct

In [5]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try: 
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,787adfc9-d37e-42f2-8efe-8e32793e0bb8
Bastion Username,kg8849_0000103312
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


## Parse the GraphML for the Topology and Create the Slice

The minidom library is used to parse the graphml. It assumes proper use of the node and edge tags. Examples of valid tags can be seen below. The name of the node is the node's id. Furthermore, it does not matter which node is the source and destination, it is parsed as an undirected graph and there will be issues if you try and create another network between the same two nodes.

    <node id="L1" />
    <node id="S1" />
    <edge source="L1" target="S1" />


In [6]:
import xml.dom.minidom
from collections import Counter

STARTING_INTF_NUM = 1

try:        
    #Create the slice
    slice = fablib.new_slice(name=SLICE_NAME)
    
    # Create dictionary to store nodes
    nodeDict = {}
    
    # Use XML parser to parse the GraphML file
    docs = xml.dom.minidom.parse(GRAPH_PATH)

    # Find all nodes via the node tag, add each to the slice with Rocky Linux as its base
    nodes = docs.getElementsByTagName("node")
    for node in nodes:
        # Grab the node name and determine if it is a client node based on its name prefix
        nodeName = node.getAttribute("id")
        isClient = True if HAS_CLIENTS and nodeName.startswith(CLIENT_PREFIX) else False # Check for compute/client nodes
        
        # Add node to the slice
        nodeInfo = slice.add_node(name=nodeName, cores=1, ram=4, image='default_rocky_8', site=SITE_NAME)
        
        # nodeDict = 0 -> FABRIC node object, 1 -> current intf number, 2 -> is a client/server (True) or not (False)
        nodeDict[nodeName] = {"nodeInfo": nodeInfo, "currentIntfNum": STARTING_INTF_NUM, "isClient": isClient}
        #[nodeInfo, 1, isClient]
        
        print(f'Added node {nodeName}')
        
        
    
    # Find all edges via the edge tag, add each to the slice via an L2Bridge connecting the node interfaces
    edges = docs.getElementsByTagName("edge")
    for edge in edges:
        # grab nodes x and y in edge (x,y)
        source = edge.getAttribute("source")
        target = edge.getAttribute("target")
        
        # Create an interface name for each interface in the network
        sourceIntfName = "eth{num}".format(node=source, num=nodeDict[source]["currentIntfNum"])
        targetIntfName = "eth{num}".format(node=target, num=nodeDict[target]["currentIntfNum"])
        
        # Name a network based on if it is a user-facing LAN (edge) or P2P links in the core of the network (core)
        networkPrefix = "edge" if nodeDict[source]["isClient"]  or nodeDict[target]["isClient"] else "core"
        networkName = f'{networkPrefix}-{source}-{target}'
        
        # Add a NIC for each node that is a part of the edge
        sourceIntf = nodeDict[source]["nodeInfo"].add_component(model='NIC_Basic', name=sourceIntfName).get_interfaces()[0]
        targetIntf = nodeDict[target]["nodeInfo"].add_component(model='NIC_Basic', name=targetIntfName).get_interfaces()[0]
        
        nodeDict[source]["currentIntfNum"] += 1
        nodeDict[target]["currentIntfNum"] += 1

        # Add a L2 network between the interfaces
        slice.add_l2network(name=networkName, interfaces=[sourceIntf, targetIntf], type="L2Bridge")
        
        print(f'Added edge {source}-{target}')


except Exception as e:
    print(f"Exception: {e}")     


Added node c1
Added node d1
Added node d2
Added node a1
Added node a2
Added node ipnode-1
Added node ipnode-2
Added edge c1-d2
Added edge c1-d1
Added edge a1-d2
Added edge d1-a2
Added edge d2-d1
Added edge a1-d1
Added edge d2-a2
Added edge a1-ipnode-1
Added edge a2-ipnode-2


## Add a Measurement Node (Optional)

Collects everything, it uses Kibana to collect data it is like a agent. Outstack is also agent.

In [7]:
if(MEAS_ADD):
    import mflib 
    print(f"MFLib version  {mflib.__version__} " )

    from mflib.mflib import MFLib

    # Add measurement node to topology using static method.
    MFLib.addMeasNode(slice, disk=100, image='docker_ubuntu_20', site=SITE_NAME)
    print("Measurement node added.")

MFLib version  1.0.4 
Measurement node added.


## Submit the Slice

In [8]:
%%time
try:
    # Submit Slice Request
    print(f'Submitting the new slice, "{SLICE_NAME}"...')
    slice.submit()
    print(f'{SLICE_NAME} creation done.')

except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()


Retry: 14, Time: 1320 sec


ID,cf3c4f6c-75b2-4566-a053-3c1a5c2e3a5d
Name,test2
Lease Expiration (UTC),2023-12-14 17:31:35 +0000
Lease Start (UTC),2023-12-13 17:31:36 +0000
Project ID,787adfc9-d37e-42f2-8efe-8e32793e0bb8
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
30b2f10d-b5d6-4ce2-b27c-3afd723cef54,a1,1,4,10,default_rocky_8,qcow2,amst-w3.fabric-testbed.net,AMST,rocky,2001:610:2d0:fabc:f816:3eff:fe93:4f70,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:610:2d0:fabc:f816:3eff:fe93:4f70,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
9907e931-f73c-4098-9397-5b4b13009e25,a2,1,4,10,default_rocky_8,qcow2,amst-w3.fabric-testbed.net,AMST,rocky,2001:610:2d0:fabc:f816:3eff:fef3:ccc0,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:610:2d0:fabc:f816:3eff:fef3:ccc0,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
f23687a5-9f49-4062-a81c-74c974546e9e,c1,1,4,10,default_rocky_8,qcow2,amst-w3.fabric-testbed.net,AMST,rocky,2001:610:2d0:fabc:f816:3eff:fe1b:77ff,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:610:2d0:fabc:f816:3eff:fe1b:77ff,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
fe500cd5-bf77-43ca-b625-82cb60bb1417,d1,1,4,10,default_rocky_8,qcow2,amst-w3.fabric-testbed.net,AMST,rocky,2001:610:2d0:fabc:f816:3eff:fe4f:a521,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:610:2d0:fabc:f816:3eff:fe4f:a521,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
66e8d20f-a65e-474c-81df-c637712bce9b,d2,1,4,10,default_rocky_8,qcow2,amst-w3.fabric-testbed.net,AMST,rocky,2001:610:2d0:fabc:f816:3eff:fe5c:ac47,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:610:2d0:fabc:f816:3eff:fe5c:ac47,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
6e2609e3-e7bd-4b41-b6a4-cca9e8a5eca5,ipnode-1,1,4,10,default_rocky_8,qcow2,amst-w3.fabric-testbed.net,AMST,rocky,2001:610:2d0:fabc:f816:3eff:fe25:5f43,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:610:2d0:fabc:f816:3eff:fe25:5f43,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
f76deb09-bea2-4604-9a8e-cf61bc278fc3,ipnode-2,1,4,10,default_rocky_8,qcow2,amst-w3.fabric-testbed.net,AMST,rocky,2001:610:2d0:fabc:f816:3eff:fec9:eeb9,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:610:2d0:fabc:f816:3eff:fec9:eeb9,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
0e78e502-c64d-4f46-9539-8d42910798bc,meas-node,4,16,100,docker_ubuntu_20,qcow2,amst-w1.fabric-testbed.net,AMST,ubuntu,2001:610:2d0:fabc:f816:3eff:fe7f:30d9,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:610:2d0:fabc:f816:3eff:fe7f:30d9,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
996e7959-b7a4-44af-8789-becc7803c2d6,core-a1-d1,L2,L2Bridge,AMST,None,None,Active,
ad11faf2-370e-4a01-a43b-acb4ad37cf52,core-a1-d2,L2,L2Bridge,AMST,None,None,Active,
1b4478f1-c69c-4118-9313-d4726681afad,core-c1-d1,L2,L2Bridge,AMST,None,None,Active,
ceb9d934-5b9b-4329-89ef-f1042dfc1cfb,core-c1-d2,L2,L2Bridge,AMST,None,None,Active,
37cc4437-5d94-4350-aec8-5903b0166fd0,core-d1-a2,L2,L2Bridge,AMST,None,None,Active,
630ab578-e9be-478d-892c-0db1a9e1023c,core-d2-a2,L2,L2Bridge,AMST,None,None,Active,
eccb2941-78a0-4d7c-b760-45da9c382efa,core-d2-d1,L2,L2Bridge,AMST,None,None,Active,
b1a3eb7f-0798-4a24-a3ac-8d4a704c3058,edge-a1-ipnode-1,L2,L2Bridge,AMST,None,None,Active,
b8e0b8dd-32e9-42b5-987d-9130c5fd2eef,edge-a2-ipnode-2,L2,L2Bridge,AMST,None,None,Active,
c81ce283-6614-4694-af02-2b7d3cf30fda,l3_meas_net_AMST,L3,FABNetv4,AMST,10.145.12.0/24,10.145.12.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
meas-node-meas_nic_meas-node_AMST-p1,p1,meas-node,l3_meas_net_AMST,100,config,,FE:B1:BA:AD:C5:16,enp7s0,enp7s0,None,6
c1-meas_nic_c1_AMST-p1,p1,c1,l3_meas_net_AMST,100,config,,1A:CD:1E:12:C1:1D,eth1,eth1,fe80::768:2c85:2082:8646,4
c1-eth1-p1,p1,c1,core-c1-d2,100,config,,1E:4C:D1:A1:15:AE,eth2,eth2,fe80::d15b:fd7b:190:b180,4
c1-eth2-p1,p1,c1,core-c1-d1,100,config,,1E:95:D4:85:92:96,eth3,eth3,fe80::9b13:298:4d82:29c3,4
d1-eth1-p1,p1,d1,core-c1-d1,100,config,,2A:EB:1E:79:B3:2A,eth4,eth4,fe80::891f:4648:3ff8:6cf7,4
d1-eth2-p1,p1,d1,core-d1-a2,100,config,,32:97:83:06:66:4E,eth5,eth5,fe80::d74b:1daa:a91c:c769,4
d1-meas_nic_d1_AMST-p1,p1,d1,l3_meas_net_AMST,100,config,,2A:12:9F:45:33:05,eth2,eth2,fe80::29a3:dd70:5918:9897,4
d1-eth4-p1,p1,d1,core-a1-d1,100,config,,2A:97:70:D2:8C:EC,eth3,eth3,fe80::5699:799a:e608:94b8,4
d1-eth3-p1,p1,d1,core-d2-d1,100,config,,26:8B:1C:BE:37:D4,eth1,eth1,fe80::666f:8ae8:dd9c:2fde,4
d2-eth4-p1,p1,d2,core-d2-a2,100,config,,6A:ED:FC:0E:01:4A,eth4,eth4,fe80::808d:2efa:b08c:4447,4



Time to print interfaces 1369 seconds
test2 creation done.
CPU times: user 3min 14s, sys: 2.46 s, total: 3min 16s
Wall time: 22min 51s


# SAVE SSH Commands

In [9]:
from FabUtils import FabOrchestrator

try:
    manager = FabOrchestrator(SLICE_NAME)
    
except Exception as e:
    print(f"Exception: {e}")
    
manager.saveSSHCommands()

Slice name: test2
Slice and nodes were acquired successfully.


## Add Basic IPv4 Addressing

In this system, 192.168.0.0/16 is the address space for all interfaces on the FABRIC slice. 

Each network is a /24 subnet of this network. Edge networks have client/compute devices with lower address (ex: .1) and networking nodes with higher addresses.

In [10]:
from ipaddress import ip_address, IPv4Address, IPv4Network

# Start with a 1 in the third octet
thirdOctet = 1

# For each newtork in the slice
for network in slice.get_networks():
    # Grab all of the usable host addresses for a new network (new third octet).
    networkAddress = f'192.168.{thirdOctet}.0/24'
    currentIPNetwork = IPv4Network(networkAddress)
    hostIPList = list(currentIPNetwork)[1:-1] # 1:-1 = remove network and broadcast address
    
    # Determine if the current network is an edge or core network
    networkName = network.get_name()
    isEdgeNetwork = True if networkName.startswith("edge") else False
    
    print(f"Configuring network {networkName} with IPv4 Network {networkAddress}")
    
    # For each interface in the network
    for intf in network.get_interfaces():
        intfName = intf.get_device_name()
        nodeName = intf.get_node().get_name()
        
        # If this is an edge network and it is a networking node, not the client node, give it a high-range address.
        if(isEdgeNetwork and not nodeDict[nodeName]["isClient"]):
            currentIPv4Address = hostIPList.pop() # Highest available host address in network
        else:
            currentIPv4Address = hostIPList.pop(0) # Lowest available host address in network
        
        # Add the address to the node
        intf.ip_addr_add(addr=currentIPv4Address, subnet=currentIPNetwork)
        print(f"\t{nodeName} {intf.get_device_name()} = {currentIPv4Address}")
        
    thirdOctet += 1

Configuring network core-d1-a2 with IPv4 Network 192.168.1.0/24
	d1 eth5 = 192.168.1.1
	a2 eth3 = 192.168.1.2
Configuring network core-d2-d1 with IPv4 Network 192.168.2.0/24
	d1 eth1 = 192.168.2.1
	d2 eth5 = 192.168.2.2
Configuring network core-a1-d1 with IPv4 Network 192.168.3.0/24
	a1 eth4 = 192.168.3.1
	d1 eth3 = 192.168.3.2
Configuring network core-d2-a2 with IPv4 Network 192.168.4.0/24
	d2 eth4 = 192.168.4.1
	a2 eth2 = 192.168.4.2
Configuring network edge-a1-ipnode-1 with IPv4 Network 192.168.5.0/24
	a1 eth3 = 192.168.5.254
	ipnode-1 eth1 = 192.168.5.1
Configuring network edge-a2-ipnode-2 with IPv4 Network 192.168.6.0/24
	a2 eth1 = 192.168.6.254
	ipnode-2 eth1 = 192.168.6.1
Configuring network l3_meas_net_AMST with IPv4 Network 192.168.7.0/24
	ipnode-2 eth2 = 192.168.7.1
	meas-node enp7s0 = 192.168.7.2
	c1 eth1 = 192.168.7.3
	d1 eth2 = 192.168.7.4
	d2 eth3 = 192.168.7.5
	a1 eth2 = 192.168.7.6
	a2 eth4 = 192.168.7.7
	ipnode-1 eth2 = 192.168.7.8
Configuring network core-c1-d2 with I

## Initalize the Measurement Framework (Optional)

Additional data collection, any kind of data stuff like CPU utlization.

In [11]:
if(MEAS_ADD):
    mf = MFLib(SLICE_NAME)

Inititializing slice "test2" for MeasurementFramework.
Found meas node as meas-node at 2001:610:2d0:fabc:f816:3eff:fe7f:30d9
Bootstrap status not found. Will now start bootstrap process...
Generating MFUser Keys...
MFUser key generation Done.
Installing mfuser account...
Installing mfuser account done.
Cloning Measurement Framework Repository from github.com...
Cloning Measurement Framework Repository from github.com done.
Configuring Measurement Network...
Measurement Network setup complete.
Generating Ansible Inventory for Measurement Framework Deployment...
Ansible Inventory for Measurement Framework Deployment generated and saved.
Bootstrapping measurement node via bash...
Starting Bootstrap Process on Measure Node (bash script)...
Bootstrap Process on Measure Node (bash script) done.
Bootstrapping measurement node via ansible...
Starting Bootstrap Process on Measure Node (Ansible Playbook)...
Bootstrap Process on Measure Node (Ansible Playbook) done.
Bootstrap ansible scripts done

#### Install Just Prometheus with Grafana

Grafana and Kibana : Visualisation agents , fabric build visualisations from Grafana and Kibana. Provides a web GUI to visualise the gathered metrics.

ELK, Prometheus - Agents to collect data 


In [12]:
%%time
instrumetize_results = mf.instrumentize( ["prometheus"] )

Instrumentizing slice "test2"
   Setting up Prometheus...
   Setting up Prometheus done.
   Setting up grafana_manager & dashboards...
   Setting up grafana_manager & dashboards done.
Instrumentize Process Complete.
CPU times: user 1.71 s, sys: 44.5 ms, total: 1.75 s
Wall time: 1min 32s


In [13]:
# Grafana SSH Tunnel Command
# mf.grafana_tunnel_local_port = 10010 # optionally change the port
print(mf.grafana_tunnel)

print(f"Browse to https://localhost:{mf.grafana_tunnel_local_port}/grafana/dashboards?query=%2A")

ssh -L 10010:localhost:443 -F ssh_config -i slice_key ubuntu@2001:610:2d0:fabc:f816:3eff:fe7f:30d9
Browse to https://localhost:10010/grafana/dashboards?query=%2A


#### Install Just ELK with Kibana

In [14]:
%%time
instrumetize_results = mf.instrumentize( ["elk"] )

Instrumentizing slice "test2"
   Setting up elk...
   Setting up elk done.
Instrumentize Process Complete.
CPU times: user 1.27 s, sys: 99.2 ms, total: 1.37 s
Wall time: 5min 17s


In [15]:
# ELK SSH Tunnel Command
# mf.kibana_tunnel_local_port = 10020 # optionally change the port
print(mf.kibana_tunnel)

print(f"Browse to http://localhost:{mf.kibana_tunnel_local_port}/")

ssh -L 10020:localhost:80 -F ssh_config -i slice_key ubuntu@2001:610:2d0:fabc:f816:3eff:fe7f:30d9
Browse to http://localhost:10020/


In [16]:
# The ELK service was created by the mf.instrumentize call.
# Get access info for Kibana by using the mflib.info call to the elk service.
# Create a dictionary to pass to the service.
data = {}
# Set the info you want to get.
data["get"] = ["nginx_id", "nginx_password"]
# Call info using service name and data dictionary.
info_results = mf.info("elk", data)
print(info_results)

if info_results["success"]:
    print(f"user: {info_results['nginx_id']} \npass: {info_results['nginx_password']}")

{'success': True, 'nginx_password': 'QMoVieVtMaefAMfLdrCh', 'nginx_id': 'fabric'}
user: fabric 
pass: QMoVieVtMaefAMfLdrCh


In [17]:
# A direct link to the overview of node data. 
print(f"http://localhost:{mf.kibana_tunnel_local_port}/app/observability/overview")

http://localhost:10020/app/observability/overview
